In [ ]:
import pandas as pd

In [ ]:
pip -q install gdown

In [ ]:
df = pd.read_csv('output_cleans.csv')
df = df[df['lang']=='en']

In [ ]:
pip install -q datasets

In [ ]:
 !pip install -q --no-cache-dir transformers sentencepiece

In [ ]:
df["rawContent"] = df["rawContent"].dropna()
df.dropna(subset=['rawContent'], how='any', inplace=True)
# df = df[20000:50000]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric
import pandas as pd

# Load the model and tokenizer
model_name = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
from tqdm import tqdm
tqdm.pandas()

# Define the sentiment labels
labels = {0: "Negative", 1: "Neutral", 2: "Positive"}


def predict_sentiment(text):
    """
    """
    encoded_input = tokenizer(text,  truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    output = model(**encoded_input)
    predictions = output.logits.argmax(-1)
    return labels[predictions.item()]

df["sentiment"] = df["rawContent"].progress_apply(predict_sentiment)

df.to_csv("tweet_sentiment.csv", index=False)

print("Sentiment analysis completed!")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric
import pandas as pd
from transformers import pipeline

# Load the model and tokenizer
model_name = f"skandavivek2/spam-classifier"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
from tqdm import tqdm
tqdm.pandas()
pipe = pipeline("text-classification", model="skandavivek2/spam-classifier")

labels = {0: "Not spam", 1: "Spam"}

# Define the function for sentiment analysis
def predict_spam(text):
    """
    Predicts the sentiment of a tweet using the model.
    """
    encoded_input = tokenizer(text,  truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    output = model(**encoded_input)
    predictions = output.logits.argmax(-1)
    return labels[predictions.item()]

df["Spam"] = df["rawContent"].progress_apply(predict_spam)


df.to_csv("spam.csv", index=False)
df
print("Spam analysis completed!")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric
import pandas as pd
from transformers import pipeline
from scipy.special import expit

# Load the model and tokenizer
model_name = f"cardiffnlp/tweet-topic-21-multi"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
from tqdm import tqdm
tqdm.pandas()
# pipe = pipeline("text-classification", model="csariyildiz/enron_spam_bert_base")

# labels = {0: "Not spam", 1: "Spam"}
class_mapping = model.config.id2label
labels = class_mapping

def predict_topic(text):
    """
    """
    encoded_input = tokenizer(text,  truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    output = model(**encoded_input)
    predictions = output.logits.argmax(-1)
    return labels[predictions.item()]

df["Topic"] = df["rawContent"].progress_apply(predict_topic)


# Save the results to a new CSV file
df.to_csv("topic.csv", index=False)
df
print("Topic analysis completed!")
